In [1]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
import time
from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

2024-11-22 17:19:24.238598: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 17:19:24.249135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732276164.262370   16768 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732276164.266073   16768 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 17:19:24.278601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model_name = "google/siglip-base-patch16-224"
# model_name = "google/siglip-so400m-patch14-224"
model = AutoModel.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

texts = ["a photo of 2 cats", "a photo of 2 dogs"]
inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
probs = torch.sigmoid(logits_per_image) # these are the probabilities
print(f"{probs[0][0]:.1%} that image 0 is '{texts[0]}'")
print(f"{probs[0][1]:.1%} that image 0 is '{texts[1]}'")

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

31.9% that image 0 is 'a photo of 2 cats'
0.0% that image 0 is 'a photo of 2 dogs'


In [3]:
model

SiglipModel(
  (text_model): SiglipTextTransformer(
    (embeddings): SiglipTextEmbeddings(
      (token_embedding): Embedding(32000, 768)
      (position_embedding): Embedding(64, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (self_attn): SiglipSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768

In [4]:
vision_model = model.vision_model
vision_model

SiglipVisionTransformer(
  (embeddings): SiglipVisionEmbeddings(
    (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
    (position_embedding): Embedding(196, 768)
  )
  (encoder): SiglipEncoder(
    (layers): ModuleList(
      (0-11): 12 x SiglipEncoderLayer(
        (self_attn): SiglipSdpaAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): SiglipMLP(
          (activation_fn): PytorchGELUTanh()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,), eps=1e-06, el

In [5]:
input_shape = (1, 3, 224, 224)  # Default SigLIP input size
dummy_input = torch.randn(input_shape)
output = vision_model(dummy_input)
output

BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.6195,  0.2554, -1.0134,  ..., -0.5341,  0.4381, -1.6009],
         [ 0.4081,  0.3242, -0.4967,  ..., -1.4876, -1.8570, -1.9226],
         [-0.7247,  1.4924, -1.0048,  ..., -2.0564,  0.2459,  0.4242],
         ...,
         [ 0.7666, -0.1956, -1.1861,  ..., -0.5558,  1.5912, -2.1294],
         [ 0.0407,  0.3529, -1.6780,  ...,  0.6625, -0.1729, -1.9680],
         [ 1.0815, -1.5693, -0.5918,  ..., -0.2207,  1.1262, -0.3552]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-3.8798e-02, -1.8350e-01,  2.8249e-01, -1.8911e-01, -1.4829e-01,
          2.2499e-02, -5.2317e-01, -4.5164e-01, -2.3904e-01,  1.8428e-01,
         -2.7591e-01, -2.3815e-01,  6.1551e-01, -6.1196e-01, -5.0202e-02,
         -4.5417e-01,  1.9159e-01,  3.3827e-02,  2.4865e-01, -1.0768e-01,
          3.9088e-01,  1.5782e-01, -1.3214e-01, -2.2701e-01, -3.8112e-01,
         -2.1870e-01,  1.0756e-01,  1.8085e-01,  2.8722e-01,  4.2217e-02,
         -3

In [6]:
output.last_hidden_state

tensor([[[ 0.6195,  0.2554, -1.0134,  ..., -0.5341,  0.4381, -1.6009],
         [ 0.4081,  0.3242, -0.4967,  ..., -1.4876, -1.8570, -1.9226],
         [-0.7247,  1.4924, -1.0048,  ..., -2.0564,  0.2459,  0.4242],
         ...,
         [ 0.7666, -0.1956, -1.1861,  ..., -0.5558,  1.5912, -2.1294],
         [ 0.0407,  0.3529, -1.6780,  ...,  0.6625, -0.1729, -1.9680],
         [ 1.0815, -1.5693, -0.5918,  ..., -0.2207,  1.1262, -0.3552]]],
       grad_fn=<NativeLayerNormBackward0>)

In [7]:
class SigLIPImageEncoder(torch.nn.Module):
    """Wrapper class for SigLIP's image encoder"""
    def __init__(self, model):
        super().__init__()
        self.vision_model = model.vision_model
        
    def forward(self, pixel_values):
        outputs = self.vision_model(pixel_values)
        # Get image embeddings - using mean pooling like in original model
        image_embeddings = outputs.last_hidden_state.mean(dim=1)
        return image_embeddings

In [8]:
image_encoder = SigLIPImageEncoder(model)

In [9]:
image_encoder

SigLIPImageEncoder(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(196, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (self_attn): SiglipSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, b

In [10]:
onn_save_path = "siglip_base_image_encoder.onnx"
torch.onnx.export(
    image_encoder,
    dummy_input,
    onn_save_path,
    export_params=True,
    opset_version=15,
    do_constant_folding=True,
    input_names=['pixel_values'],
    output_names=['image_embeddings'],
    dynamic_axes={
        'pixel_values': {0: 'batch_size'},
        'image_embeddings': {0: 'batch_size'}
    }
)